In [ ]:
# download & import libraries
!pip install --upgrade pip
!pip install bs4
!pip install pandas
!pip install requests
!pip install numpy
!pip install urllib3
!pip install phonenumbers
!pip install py3-validate-email
!pip install fuzzywuzzy

from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib
from urllib.parse import urlparse
import numpy as np
import phonenumbers
from validate_email import validate_email
from fuzzywuzzy.fuzz import partial_ratio
import csv
import json
import os

In [ ]:
# timestamps of gvmt certified test providers
urls = [ 
    'https://web.archive.org/web/20210519121145/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210520140910/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210526072031/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210529103556/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210531150239/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210604100539/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210618121419/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210626230824/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210629104343/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210701142411/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210708123655/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210713165043/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210729180818/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210808132243/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210811073050/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210812113842/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210817114915/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210818174329/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210823032256/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://web.archive.org/web/20210824065330/https://www.find-travel-test-provider.service.gov.uk/test-type/amber',
    'https://www.find-travel-test-provider.service.gov.uk/test-type/amber'
]

provider_details = pd.DataFrame(columns=['company_name','company_link', 'company_number', 'company_email', 'price'])

for url in urls:
    print(url)
    # scrape provider table for each time stamp
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    providers_table = soup.find('table', {'class', 'govuk-table'}).find('tbody')

    url_provider_details = pd.DataFrame(columns=['company_name', 'company_link', 'company_number', 'company_email'])
    for row in providers_table.find_all('tr'):
        # test provider saved in cell with id 'provider'
        provider = row.find(id='provider').find('a')

        # scrape name and link of provider
        name = provider.get_text().rstrip().lower()
        link = provider['href']
    
        # test provider number and email saved in only cell(s) with no id
        number_email = row.find_all('td', id=None)
        
        # remove web archive prefix from wayback machine
        link = link[43:] if 'web.archive.org' in link else link

        # old format stores number and email in separate cells
        # new format stores number and email in same cell
        if len(number_email) == 1:
            number_email = number_email[0].find_all('a')
        number = str(number_email[0].get_text())
        email = number_email[1].get_text()

        # test provider price saved in cell with id 'priceAmber'
        price = float(row.find(id='priceAmber').get_text().replace('£', ''))
        
        # clean scraped data
        if number and len(number) >= 10:
            number = ' '.join(number.rstrip().split())
            number = phonenumbers.format_number(phonenumbers.parse(number, 'GB'), phonenumbers.PhoneNumberFormat.INTERNATIONAL)
        else:
            number = np.nan 
        if email:
            email = str(email).rstrip().replace('\n', '').lower()
        if link:
            link = urlparse(link.lower()).netloc

        # add provider details to df
        url_provider_details = url_provider_details.append({
            'company_name': name,
            'company_link': link,
            'company_number': number,
            'company_email': email,
            'price': price
        }, ignore_index=True)

    # merge provider details across urls
    provider_details = pd.merge(
        provider_details, 
        url_provider_details, 
        how="outer", 
        on=['company_name','company_link','company_number','company_email', 'price']
    )

provider_details.to_csv('./datasets/aggregated-details/provider_details.csv', index=False)

In [99]:
# group all numbers, emails, and links for each company together
grouped_details = pd.DataFrame(columns=['company_name'])
provider_details = pd.read_csv('./datasets/aggregated-details/provider_details.csv')

for category in ['company_number', 'company_email', 'company_link', 'price']:
    # remove duplicate scraped providers
    category_details = provider_details[['company_name', category]].drop_duplicates()

    # group providers together by name (i.e. group all given prices/emails/number for a provider)
    df = (category_details.set_index(['company_name', category_details.groupby('company_name').cumcount()])[category]
            .unstack(fill_value='')
            .add_prefix(category+'_')
            .reset_index())
    grouped_details = pd.merge(grouped_details, df, how='outer',on=['company_name'])

grouped_details.to_csv('./datasets/aggregated-details/grouped_details.csv', index=False)

In [ ]:
# produce risk score based on price data
grouped_price = pd.read_csv('./datasets/aggregated-details/grouped_details.csv')
price_cols = [x for x in grouped_price if 'price' in x]

price_details = pd.DataFrame(columns=['num_matches', 'num_jumps', 'max_change'])
for company in grouped_price['company_name']:
    print(company)
    # match identified providers with similar names 
    fuzzy_matches = [ind for ind in grouped_price.index if partial_ratio(company, grouped_price['company_name'][ind])>=90]

    matched_rows = grouped_price.loc[fuzzy_matches][price_cols]
    mean_prices = matched_rows.mean(axis=0).dropna() # average across similar providers

    num_jumps = max_change = 0
    prev_price = mean_prices[0]
    for price in mean_prices[1:]:
        # track max price change and total price changes
        if prev_price != price:
            num_jumps += 1
            change = abs((price - prev_price)/prev_price)

            if change > max_change:
                max_change = change
        
        prev_price = price

    # add price signals to df
    price_details = price_details.append({
        'num_matches': len(fuzzy_matches), 
        'num_jumps': num_jumps,
        'max_change': max_change*100
    }, ignore_index=True)


price_details['company_name'] = grouped_price['company_name']
price_details.to_csv('./datasets/indicators/price_details.csv', index=False)

In [ ]:
# validate scraped phone numbers
grouped_details = pd.read_csv('./datasets/aggregated-details/grouped_details.csv')
phone_cols = [x for x in grouped_details.columns if 'company_number' in x]

def check_number(x):
    results = []
    for cell in x:
        if pd.isna(cell) or len(str(cell)) < 9:
            results.append('')
        else:
            number = phonenumbers.parse(cell,'GB')
            valid = phonenumbers.is_valid_number(number)
            results.append(bool(valid))
    return results

phone_details = grouped_details[phone_cols]
phone_validated = phone_details.apply(check_number)
invalid_number = pd.DataFrame({
    'company_name': grouped_details['company_name'], 
    'phone_invalid': [False]*len(grouped_details['company_name'])
})

invalid_cols = phone_validated[phone_validated.eq(False).any(axis=1)].index
empty_cols = phone_validated[phone_validated.eq('').all(axis=1)].index

invalid_number['phone_invalid'].iloc[invalid_cols] = True
invalid_number['phone_invalid'].iloc[empty_cols] = True

invalid_number.to_csv('./datasets/indicators/phone_validation.csv', index=False)

In [ ]:
# validate scraped email addresses
grouped_details = pd.read_csv('./datasets/aggregated-details/grouped_details.csv')
email_cols = [x for x in grouped_details.columns if 'company_email' in x]

def check_email(x):
    results = []
    for cell in x:
        print(str(cell).rstrip())
        if pd.isna(cell) or not cell:
            results.append('')
        else:
            valid = validate_email(email_address=str(cell).rstrip())
            results.append(bool(valid))
    return results
    

email_details = grouped_details[email_cols]
email_validated = email_details.apply(check_email)
invalid_email = pd.DataFrame({
    'company_name': grouped_details['company_name'], 
    'email_invalid': [False]*len(grouped_details['company_name'])
})


invalid_cols = email_validated[email_validated.eq(False).any(axis=1)].index
empty_cols = email_validated[email_validated.eq('').all(axis=1)].index

invalid_email['email_invalid'].iloc[invalid_cols] = True
invalid_email['email_invalid'].iloc[empty_cols] = True

invalid_email.to_csv('./datasets/indicators/email_validation.csv', index=False)


In [ ]:
# obtaining trustpilot review scores
grouped_details = pd.read_csv('./datasets/aggregated-details/grouped_details.csv')
link_cols = [x for x in grouped_details.columns if 'company_link' in x]

def check_trustpilot(x):
    results = []
    print(len(x))
    for cell in x:
        if pd.isna(cell) or not cell:
            results.append(np.nan)
        else:
            # scrape trust pilot score for each company
            page = requests.get("https://uk.trustpilot.com/review/"+cell)
            print(page)
            soup = BeautifulSoup(page.content, "html.parser")
            score = soup.find('p', {'class', 'header_trustscore'})
                
            results.append(float(score.get_text()) if score else np.nan)

    return results


link_details = grouped_details[link_cols]
trustpilot_scores = link_details.apply(check_trustpilot)
trustpilot_scores[trustpilot_scores.eq(0)] = np.nan 

trustpilot_scores['trustpilot_score'] = trustpilot_scores.mean(axis=1)

# categorise scores
trustpilot_scores['score_category'] = ['']*len(grouped_details)
trustpilot_scores['score_category'].iloc[trustpilot_scores[trustpilot_scores['trustpilot_score'].isna()].index] = "No Score"
trustpilot_scores['score_category'].iloc[trustpilot_scores[trustpilot_scores['trustpilot_score'].between(0, 1, inclusive="right")].index] = "0-1"
trustpilot_scores['score_category'].iloc[trustpilot_scores[trustpilot_scores['trustpilot_score'].between(1, 2, inclusive="right")].index] = "1-2"
trustpilot_scores['score_category'].iloc[trustpilot_scores[trustpilot_scores['trustpilot_score'].between(2, 3, inclusive="right")].index] = "2-3"
trustpilot_scores['score_category'].iloc[trustpilot_scores[trustpilot_scores['trustpilot_score'].between(3, 4, inclusive="right")].index] = "3-4"
trustpilot_scores['score_category'].iloc[trustpilot_scores[trustpilot_scores['trustpilot_score'].between(4, 5, inclusive="right")].index] = "4-5"

trustpilot_scores['is_reviewed'] = [True]*len(grouped_details)
trustpilot_scores['is_reviewed'].iloc[trustpilot_scores[trustpilot_scores['trustpilot_score'].isna()].index] = False

trustpilot_scores['company_name'] = grouped_details['company_name']
trustpilot_scores[['trustpilot_score', 'score_category', 'is_reviewed', 'company_name']].to_csv('./datasets/indicators/trustpilot_scores.csv', index=False)

In [ ]:
# match registered CH companies
api_key = '3511f785-bc60-40b3-b697-36bedb63c848'

grouped_details = pd.read_csv('./datasets/aggregated-details/grouped_details.csv')
matched_companies = pd.DataFrame(columns=['company_name', 'registered_name'])
matched_companies['company_name'] = grouped_details['company_name']

matched = []
for company in grouped_details['company_name']:
    match = ''

    company = company.lower()
    response = requests.get(
        'https://api.company-information.service.gov.uk/search?items_per_page=1&q='+'+'.join(company.split()), 
        auth=(api_key, '')
    )
    print(response)

    # if top company found on CH is similar to company name consider it registsered
    if response:
        response_json = response.json()
        if response_json['total_results'] > 1:
            company_found = response.json()['items'][0]['title'].lower()

            if partial_ratio(company_found, company) >= 80:
                match = company_found
        
    matched.append(match)

matched_companies['registered_name'] = matched
matched_companies['is_registered'] = [True]*len(matched_companies)
matched_companies['is_registered'].iloc[matched_companies['registered_name'].eq('')] = False

matched_companies.to_csv('./datasets/indicators/registered.csv', index=False)

In [ ]:
root = "./datasets/indicators/"

# merge indicators 
merged_df = pd.DataFrame()
first = True
for file in os.listdir(root):
    print(file)
    metric_df = pd.read_csv(root + file)

    if first:
        merged_df['company_name'] = metric_df['company_name']
        first = False

    metric_df = metric_df.drop(metric_df[~metric_df['company_name'].isin(merged_df['company_name'])] .index)
    merged_df = pd.merge(merged_df, metric_df, on='company_name', how="outer")

merged_df.to_csv('./datasets/overall_indicators.csv')

In [ ]:
# open indicators
with open("./datasets/overall_indicators.csv", "r") as f:
    overall_indicators = list(csv.DictReader(f))

# clustering algorithm API call
data = {
    "Inputs": {
        "WebServiceInput0": overall_indicators,
    }, "GlobalParameters": {
}
}
 
body = str.encode(json.dumps(data))
 
url = 'http://9db7173e-c2f5-4b4c-8fb8-e73cf648193c.uksouth.azurecontainer.io/score'
api_key = 'dcY6cVNnsdQQp1hLzVUPtXtWYKe5bpBY'
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
 
req = urllib.request.Request(url, body, headers)

# parse API call response to obtain cluster assignments
try:
    response = urllib.request.urlopen(req)
 
    result = response.read().decode("utf-8") 
    result_json = json.loads(result)['Results']['WebServiceOutput0']
    
    companies_clustered = pd.DataFrame(result_json)
    companies_clustered.to_csv('./companies_clustered.csv', index=False)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
 
    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

In [ ]:
# calculate risk score and add to overall_indicators
merged_df.read_csv('./datasets/overall_indicators.csv')

merged_df['risk_score'] = (1-merged_df['Assignments']) + merged_df['email_invalid'] + merged_df['phone_invalid'] + (5-merged_df['trustpilot_score'].fillna(0))
merged_df.sort_values(by='risk_score', ascending=False)
merged_df.to_csv('./datasets/overall_indicators.csv', index=False)